# Human-in-the-Loop-työnkulku Microsoft Agent Frameworkilla

## 🎯 Oppimistavoitteet

Tässä notebookissa opit toteuttamaan **human-in-the-loop**-työnkulkuja Microsoft Agent Frameworkin `RequestInfoExecutor`-toiminnon avulla. Tämä tehokas malli mahdollistaa AI-työnkulkujen keskeyttämisen ihmisen syötteen keräämiseksi, tehden agenteista vuorovaikutteisia ja antaen ihmisille hallinnan kriittisissä päätöksissä.

## 🔄 Mitä on Human-in-the-Loop?

**Human-in-the-loop (HITL)** on suunnittelumalli, jossa AI-agentit keskeyttävät suorituksen pyytääkseen ihmisen syötettä ennen jatkamista. Tämä on tärkeää seuraavissa tilanteissa:

- ✅ **Kriittiset päätökset** - Hanki ihmisen hyväksyntä ennen tärkeiden toimien suorittamista
- ✅ **Epäselvät tilanteet** - Anna ihmisten selventää, kun AI on epävarma
- ✅ **Käyttäjän mieltymykset** - Pyydä käyttäjiä valitsemaan useiden vaihtoehtojen välillä
- ✅ **Sääntely ja turvallisuus** - Varmista ihmisen valvonta säädellyissä operaatioissa
- ✅ **Vuorovaikutteiset kokemukset** - Rakenna keskusteluagentteja, jotka reagoivat käyttäjän syötteisiin

## 🏗️ Miten se toimii Microsoft Agent Frameworkissa

Framework tarjoaa kolme keskeistä komponenttia HITL:lle:

1. **`RequestInfoExecutor`** - Erityinen suorittaja, joka keskeyttää työnkulun ja lähettää `RequestInfoEvent`-tapahtuman
2. **`RequestInfoMessage`** - Perusluokka tyypitetyille pyyntöviesteille, jotka lähetetään ihmisille
3. **`RequestResponse`** - Yhdistää ihmisen vastaukset alkuperäisiin pyyntöihin käyttäen `request_id`-tunnistetta

**Työnkulun malli:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Esimerkkimme: Hotellivaraus käyttäjän vahvistuksella

Rakennamme ehdollisen työnkulun pohjalta lisäämällä ihmisen vahvistuksen **ennen** vaihtoehtoisten kohteiden ehdottamista:

1. Käyttäjä pyytää kohdetta (esim. "Pariisi")
2. `availability_agent` tarkistaa, onko huoneita saatavilla
3. **Jos huoneita ei ole** → `confirmation_agent` kysyy "Haluatko nähdä vaihtoehtoja?"
4. Työnkulku **keskeytyy** käyttäen `RequestInfoExecutor`-toimintoa
5. **Ihminen vastaa** "kyllä" tai "ei" konsolisyötteen kautta
6. `decision_manager` ohjaa vastauksen perusteella:
   - **Kyllä** → Näytä vaihtoehtoiset kohteet
   - **Ei** → Peru varauspyyntö
7. Näytä lopputulos

Tämä esimerkki osoittaa, kuinka käyttäjille voidaan antaa hallinta agentin ehdotuksista!

---

Aloitetaan! 🚀


## Vaihe 1: Tuo tarvittavat kirjastot

Tuomme standardin Agent Framework -komponentit sekä **ihmisen osallistumiseen liittyvät luokat**:
- `RequestInfoExecutor` - Suorittaja, joka keskeyttää työnkulun ihmisen syötettä varten
- `RequestInfoEvent` - Tapahtuma, joka lähetetään, kun ihmisen syötettä pyydetään
- `RequestInfoMessage` - Perusluokka tyypitetyille pyyntöjen tiedoille
- `RequestResponse` - Yhdistää ihmisen vastaukset pyyntöihin
- `WorkflowOutputEvent` - Tapahtuma työnkulun tulosten havaitsemiseksi


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Vaihe 2: Määritä Pydantic-mallit jäsennellyille tulosteille

Nämä mallit määrittävät **skeeman**, jonka agentit palauttavat. Säilytämme kaikki ehdollisen työnkulun mallit ja lisäämme:

**Uutta ihmisen osallistumiselle:**
- `HumanFeedbackRequest` - `RequestInfoMessage`-luokan aliluokka, joka määrittää ihmisille lähetettävän pyynnön sisällön
  - Sisältää `prompt` (kysymys, joka esitetään) ja `destination` (konteksti koskien saatavilla olevaa kaupunkia)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Vaihe 3: Luo hotellivaraustyökalu

Sama työkalu kuin ehdollisessa työnkulussa - tarkistaa, ovatko huoneet saatavilla kohteessa.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Vaihe 4: Määritä reitityksen ehtofunktiot

Tarvitsemme **neljä ehtofunktiota** ihmisen mukanaoloa vaativaan työnkulkuun:

**Ehdollisesta työnkulusta:**
1. `has_availability_condition` - Reitittää, kun hotelleja ON saatavilla
2. `no_availability_condition` - Reitittää, kun hotelleja EI ole saatavilla

**Uutta ihmisen mukanaoloa varten:**
3. `user_wants_alternatives_condition` - Reitittää, kun käyttäjä vastaa "kyllä" vaihtoehdoille
4. `user_declines_alternatives_condition` - Reitittää, kun käyttäjä vastaa "ei" vaihtoehdoille


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Vaihe 5: Luo Decision Manager Executor

Tämä on **ihmisen mukanaolon mallin ydin**! `DecisionManager` on mukautettu `Executor`, joka:

1. **Vastaanottaa ihmisen palautetta** `RequestResponse`-objektien kautta
2. **Käsittelee käyttäjän päätöksen** (kyllä/ei)
3. **Ohjaa työnkulun** lähettämällä viestejä sopiville agenteille

Keskeiset ominaisuudet:
- Käyttää `@handler`-koristetta menetelmien esittämiseen työnkulun vaiheina
- Vastaanottaa `RequestResponse[HumanFeedbackRequest, str]`, joka sisältää sekä alkuperäisen pyynnön että käyttäjän vastauksen
- Tuottaa yksinkertaisia "kyllä" tai "ei" -viestejä, jotka käynnistävät ehtofunktiomme


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Vaihe 6: Luo mukautettu näyttöohjelma

Sama näyttöohjelma kuin ehdollisessa työnkulussa - tuottaa lopulliset tulokset työnkulun tuloksena.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Vaihe 7: Lataa ympäristömuuttujat

Määritä LLM-asiakas (GitHub Models, Azure OpenAI tai OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Vaihe 8: Luo tekoälyagentteja ja suorittajia

Luomme **kuusi työnkulun komponenttia**:

**Agentit (käärittynä AgentExecutoriin):**
1. **availability_agent** - Tarkistaa hotellien saatavuuden työkalun avulla
2. **confirmation_agent** - 🆕 Valmistelee vahvistuspyynnön ihmiselle
3. **alternative_agent** - Ehdottaa vaihtoehtoisia kaupunkeja (kun käyttäjä sanoo kyllä)
4. **booking_agent** - Kannustaa varaamaan (kun huoneita on saatavilla)
5. **cancellation_agent** - 🆕 Käsittelee peruutusviestin (kun käyttäjä sanoo ei)

**Erikoissuorittajat:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor`, joka keskeyttää työnkulun ihmisen syötettä varten
7. **decision_manager** - 🆕 Mukautettu suorittaja, joka ohjaa ihmisen vastauksen perusteella (määritelty jo yllä)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Vaihe 9: Työnkulun rakentaminen ihmisen osallistumisella

Nyt rakennamme työnkulun kaavion, jossa on **ehdollinen reititys**, mukaan lukien ihmisen osallistumisen polku:

**Työnkulun rakenne:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Keskeiset reitit:**
- `availability_agent → confirmation_agent` (kun huoneita ei ole)
- `confirmation_agent → prepare_human_request` (muuntaa tyypin)
- `prepare_human_request → request_info_executor` (tauko ihmistä varten)
- `request_info_executor → decision_manager` (aina - tarjoaa RequestResponse)
- `decision_manager → alternative_agent` (kun käyttäjä sanoo "kyllä")
- `decision_manager → cancellation_agent` (kun käyttäjä sanoo "ei")
- `availability_agent → booking_agent` (kun huoneita on saatavilla)
- Kaikki polut päättyvät `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Vaihe 10: Suorita testitapaus 1 - Kaupunki ILMAN saatavuutta (Pariisi ihmisen vahvistuksella)

Tämä testi havainnollistaa **koko ihmisen mukanaolon syklin**:

1. Pyydä hotellia Pariisista
2. availability_agent tarkistaa → Ei huoneita
3. confirmation_agent luo ihmisläheisen kysymyksen
4. request_info_executor **keskeyttää työnkulun** ja lähettää `RequestInfoEvent`-tapahtuman
5. **Sovellus havaitsee tapahtuman ja kehottaa käyttäjää konsolissa**
6. Käyttäjä kirjoittaa "kyllä" tai "ei"
7. Sovellus lähettää vastauksen `send_responses_streaming()`-toiminnon kautta
8. decision_manager ohjaa vastauksen perusteella
9. Lopputulos näytetään

**Keskeinen toimintamalli:**
- Käytä `workflow.run_stream()` ensimmäisellä kierroksella
- Käytä `workflow.send_responses_streaming(pending_responses)` seuraavilla kierroksilla
- Kuuntele `RequestInfoEvent`-tapahtumaa havaitaksesi, milloin tarvitaan ihmisen syöttöä
- Kuuntele `WorkflowOutputEvent`-tapahtumaa saadaksesi lopulliset tulokset


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Vaihe 11: Suorita testitapaus 2 - Kaupunki, jossa saatavuus (Tukholma - Ei ihmisen syötettä tarvittu)

Tämä testi havainnollistaa **suoran polun**, kun huoneita on saatavilla:

1. Pyydä hotellia Tukholmasta
2. availability_agent tarkistaa → Huoneita saatavilla ✅
3. booking_agent ehdottaa varausta
4. display_result näyttää vahvistuksen
5. **Ei ihmisen syötettä tarvittu!**

Työnkulku ohittaa kokonaan ihmisen mukanaolon polun, kun huoneita on saatavilla.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Keskeiset opit ja ihmisen mukanaolon parhaat käytännöt

### ✅ Mitä olet oppinut:

#### 1. **RequestInfoExecutor-malli**
Microsoft Agent Frameworkin ihmisen mukanaolon malli käyttää kolmea keskeistä komponenttia:
- `RequestInfoExecutor` - Keskeyttää työnkulun ja lähettää tapahtumia
- `RequestInfoMessage` - Perusluokka tyypitetyille pyyntöjen sisällöille (aliluokita tämä!)
- `RequestResponse` - Yhdistää ihmisen vastaukset alkuperäisiin pyyntöihin

**Keskeinen ymmärrys:**
- `RequestInfoExecutor` EI kerää syötettä itse - se vain keskeyttää työnkulun
- Sovelluskoodisi täytyy kuunnella `RequestInfoEvent`-tapahtumaa ja kerätä syötteet
- Sinun täytyy kutsua `send_responses_streaming()` sanakirjalla, joka yhdistää `request_id`:n käyttäjän vastaukseen

#### 2. **Suoratoiston toteutusmalli**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Tapahtumapohjainen arkkitehtuuri**
Kuuntele tiettyjä tapahtumia työnkulun ohjaamiseksi:
- `RequestInfoEvent` - Tarvitaan ihmisen syötettä (työnkulku keskeytetty)
- `WorkflowOutputEvent` - Lopputulos saatavilla (työnkulku valmis)
- `WorkflowStatusEvent` - Tilamuutokset (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS jne.)

#### 4. **Mukautetut suorittimet @handlerilla**
`DecisionManager` näyttää, kuinka luoda suorittimia, jotka:
- Käyttävät `@handler`-koristetta metodien altistamiseen työnkulun vaiheina
- Vastaanottavat tyypitetyjä viestejä (esim. `RequestResponse[HumanFeedbackRequest, str]`)
- Ohjaavat työnkulun lähettämällä viestejä muille suorittimille
- Pääsevät käsiksi kontekstiin `WorkflowContext`in kautta

#### 5. **Ehdollinen ohjaus ihmisen päätöksillä**
Voit luoda ehtofunktioita, jotka arvioivat ihmisen vastauksia:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Käytännön sovellukset:

1. **Hyväksymistyönkulut**
   - Hanki esimiehen hyväksyntä ennen kuluraporttien käsittelyä
   - Vaadi ihmisen tarkistus ennen automaattisten sähköpostien lähettämistä
   - Vahvista suurten arvojen tapahtumat ennen toteutusta

2. **Sisällön moderointi**
   - Merkitse kyseenalainen sisältö ihmisen tarkistettavaksi
   - Pyydä moderaattoreita tekemään lopullinen päätös rajatapauksissa
   - Siirrä ihmisille, kun tekoälyn varmuus on alhainen

3. **Asiakaspalvelu**
   - Anna tekoälyn käsitellä rutiinikysymyksiä automaattisesti
   - Siirrä monimutkaiset ongelmat ihmisten käsiteltäväksi
   - Kysy asiakkaalta, haluaako hän puhua ihmisen kanssa

4. **Tietojen käsittely**
   - Pyydä ihmisiä ratkaisemaan epäselvät tietomerkinnät
   - Vahvista tekoälyn tulkinnat epäselvistä asiakirjoista
   - Anna käyttäjien valita useiden pätevien tulkintojen välillä

5. **Turvallisuuskriittiset järjestelmät**
   - Vaadi ihmisen vahvistus ennen peruuttamattomia toimia
   - Hanki hyväksyntä ennen arkaluontoisten tietojen käyttöä
   - Vahvista päätökset säännellyillä aloilla (terveydenhuolto, rahoitus)

6. **Interaktiiviset agentit**
   - Rakenna keskustelubotteja, jotka esittävät jatkokysymyksiä
   - Luo ohjeita, jotka opastavat käyttäjiä monimutkaisten prosessien läpi
   - Suunnittele agentteja, jotka tekevät yhteistyötä ihmisten kanssa vaihe vaiheelta

### 🔄 Vertailu: Ilman vs ihmisen mukanaolo

| Ominaisuus | Ehdollinen työnkulku | Ihmisen mukanaolon työnkulku |
|------------|----------------------|------------------------------|
| **Suoritus** | Yksi `workflow.run()` | Silmukka `run_stream()` + `send_responses_streaming()` |
| **Käyttäjän syöte** | Ei mitään (täysin automatisoitu) | Interaktiiviset kehotteet `input()` tai käyttöliittymän kautta |
| **Komponentit** | Agentit + Suorittimet | + RequestInfoExecutor + DecisionManager |
| **Tapahtumat** | Vain AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent jne. |
| **Keskeytys** | Ei keskeytystä | Työnkulku keskeytyy RequestInfoExecutorissa |
| **Ihmisen hallinta** | Ei ihmisen hallintaa | Ihmiset tekevät keskeisiä päätöksiä |
| **Käyttötarkoitus** | Automaatio | Yhteistyö ja valvonta |

### 🚀 Edistyneet mallit:

#### Useita ihmisen päätöspisteitä
Voit käyttää useita `RequestInfoExecutor`-solmuja samassa työnkulussa:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Aikarajojen käsittely
Toteuta aikarajat ihmisen vastauksille:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Rikas käyttöliittymäintegraatio
Käytä `input()`-komennon sijaan verkkokäyttöliittymää, Slackia, Teamsia jne.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Ehdollinen ihmisen mukanaolo
Pyydä ihmisen syötettä vain tietyissä tilanteissa:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Parhaat käytännöt:

1. **Aliluokita aina RequestInfoMessage**
   - Tarjoaa tyyppiturvallisuutta ja validointia
   - Mahdollistaa rikkaan kontekstin käyttöliittymän renderöintiin
   - Selkeyttää kunkin pyyntötyypin tarkoitusta

2. **Käytä kuvailevia kehotteita**
   - Sisällytä konteksti siitä, mitä kysyt
   - Selitä kunkin valinnan seuraukset
   - Pidä kysymykset yksinkertaisina ja selkeinä

3. **Käsittele odottamaton syöte**
   - Vahvista käyttäjän vastaukset
   - Tarjoa oletusarvot virheelliselle syötteelle
   - Anna selkeitä virheilmoituksia

4. **Seuraa pyyntöjen tunnisteita**
   - Käytä yhteyttä request_id:n ja vastausten välillä
   - Älä yritä hallita tilaa manuaalisesti

5. **Suunnittele ei-blokkaavaksi**
   - Älä estä säikeitä odottamalla syötettä
   - Käytä asynkronisia malleja kauttaaltaan
   - Tue samanaikaisia työnkulun instansseja

### 📚 Liittyvät käsitteet:

- **Agent Middleware** - Sieppaa agenttikutsut (edellinen muistikirja)
- **Työnkulun tilanhallinta** - Säilytä työnkulun tila ajokertojen välillä
- **Moniagenttinen yhteistyö** - Yhdistä ihmisen mukanaolo agenttitiimeihin
- **Tapahtumapohjaiset arkkitehtuurit** - Rakenna reaktiivisia järjestelmiä tapahtumilla

---

### 🎓 Onnittelut!

Olet oppinut hallitsemaan ihmisen mukanaolon työnkulkuja Microsoft Agent Frameworkilla! Nyt osaat:
- ✅ Keskeyttää työnkulkuja ihmisen syötteen keräämiseksi
- ✅ Käyttää RequestInfoExecutor- ja RequestInfoMessage-komponentteja
- ✅ Käsitellä suoratoiston toteutusta tapahtumilla
- ✅ Luoda mukautettuja suorittimia @handlerilla
- ✅ Ohjata työnkulkuja ihmisen päätösten perusteella
- ✅ Rakentaa interaktiivisia tekoälyagentteja, jotka tekevät yhteistyötä ihmisten kanssa

**Tämä on kriittinen malli luotettavien ja hallittavien tekoälyjärjestelmien rakentamiseen!** 🚀



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäinen asiakirja sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Tärkeissä tiedoissa suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
